# Complete guide

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



import trecs
from trecs.models import ContentFiltering, ImplicitMF
from trecs.random import Generator
from trecs.metrics import MSEMeasurement, RecSimilarity, InteractionSimilarity, RecallMeasurement
from trecs.components import Users
import trecs.matrix_ops as mo #Note, in order for the ideal recommender to result in non-insane results, the normalize_users parameter in mo.inner_product must be set to False

In [ ]:
#https://web.ma.utexas.edu/users/mks/statmistakes/skeweddistributions.html

In [2]:
class IdealRecommender(ContentFiltering):
    """
    With the Ideal Recommender, we make the *strong assumption* that the true scores are provided
    to the recommender system through a custom scoring function, which always returns the true
    underlying user-item scores. Therefore, this class is pretty much an empty skeleton; the only
    modification is that we don't update any internal state of the recommender at each time step.
    
    Amy: Note to self, as a part of the baseline ContentFiltering class, if an actual_item_representation 
    """
    def __init__(self, *args, **kwargs):
        
        
        super().__init__(*args, **kwargs)
    

    def _update_internal_state(self, interactions):
        # do not change users_hat!
        pass


In [3]:
#np.random.seed(1)
plt.style.use('seaborn')
plt.rcParams.update({'font.size': 50})

In [12]:
# Keep the dimensions small for easy visualization
number_of_users = 5
number_of_attributes = 10
number_of_items = 15
# We define user_representation using the standard integer generator in Numpy.
# We assume a number of interactions with each attribute in the interval [0,4).

users_shape = (number_of_users, number_of_attributes)
actual_user_representation = Users(size=users_shape, num_users=number_of_users)
model_user_representation = np.random.randint(4, size=(number_of_users, number_of_attributes))

# We define item_representation using the Generator that comes with the framework
# We assume a binary matrix with a binomial distribution

actual_item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)

model_item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)

In [ ]:
#Not positive this is correct

content = ContentFiltering(
    num_users=number_of_users,
    num_items=number_of_items,
    num_attributes=number_of_attributes,
    actual_user_representation=actual_user_representation,
    user_representation=model_user_representation, 
    actual_item_representation = actual_item_representation,
    item_representation=actual_item_representation, #model has the true item values
)
# add an MSE measurement
content.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
content.run(timesteps=5)

In [ ]:
ideal = IdealRecommender(
    num_users=number_of_users,
    num_items=number_of_items,
    num_attributes=number_of_attributes,
    actual_user_representation=actual_user_representation,
    user_representation=actual_user_representation.actual_user_profiles.value, 
    actual_item_representation = actual_item_representation,
    item_representation=actual_item_representation, #model has the true item values
    num_items_per_iter="all",
)
# add an MSE measurement
ideal.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
ideal.run(timesteps=5)

In [13]:
mf = ImplicitMF(
    num_users=number_of_users,
    num_items=number_of_items,
    num_latent_factors=number_of_attributes,
    actual_user_representation=actual_user_representation,
    actual_item_representation = actual_item_representation,
    num_items_per_iter="all",
)
# add an MSE measurement
mf.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
#mf.run(timesteps=5,train_between_steps=True)

In [14]:
mf.run(timesteps=1)

100%|██████████| 1/1 [00:00<00:00, 496.54it/s]


In [18]:
import math

In [15]:
k=10
recommender=mf

shown_item_rel = np.take(recommender.predicted_scores.value, recommender.items_shown)
shown_item_ranks = np.argsort(shown_item_rel, axis=1)
top_k_items = np.take(recommender.items_shown, shown_item_ranks[:, k :])

#np.log2()

In [55]:
#recommender.actual_user_item_scores.shape
#recommender.predicted_scores.value.shape
#print(recommender.items_shown)

#print(recommender.items_shown.shape[1])


#c = np.tile(np.array(np.arange(0,recommender.num_items), (recommender.num_users,1)))

#np.tile(c,(4,1))
perfect_item_rel = recommender.actual_user_item_scores
ideal_ranks = np.tile(np.arange(0,recommender.items_shown.shape[1]), (recommender.num_users,1))

idcg = recommender.actual_user_item_scores / np.log2(ideal_item_ranks+2)

print(idcg)

NameError: name 'ideal_item_ranks' is not defined

In [35]:
#perfect_items = np.take(recommender.actual_user_item_scores, recommender.items_shown)
#print(perfect_items)



perfect_item_ranks = np.argsort(recommender.actual_user_item_scores, axis=1)
perfect_item_ranks_shown = np.take(perfect_item_ranks, recommender.items_shown)
print(perfect_item_ranks)

print(perfect_item_ranks_shown)


[[ 3  8 10 13  2  9 12  0  5  6  1  7  4 11 14]
 [11  9 12  4  6  2  7  1 10 14 13  5  3  8  0]
 [ 1  2 10  7  4 14 11  9 13  0  5  6 12  8  3]
 [ 4  7  9 11 13 14  2 12  1 10  6  0  8  3  5]
 [ 5  6  0  7 12  4  1 14  8  3 10 11 13  2  9]]
[[ 7  9  4  8  0  1  5 10  2 11  3  6 13 14 12]
 [ 4  8 13 12  7 11 14  0  6 10  3  1  9  5  2]
 [ 6  1  5  2  9  3  4 11 12 14  0  8 10  7 13]
 [ 6  4  8  2  1  0  9  7  3 14 10 13 12  5 11]
 [ 6 11  8  7  4 14 12  0  9 10 13  3  2  1  5]]


In [ ]:
idcg = shown_item_rel

In [23]:
print(shown_item_rel[0])
print(shown_item_ranks)
#print(shown_item_ranks+2)


dcg = shown_item_rel / np.log2(shown_item_ranks+2)
print(dcg)

[ 1.70670957  1.70153075  1.04025762  0.89849223  0.84173072  0.50763789
  0.4698678   0.43541279  0.37780654  0.04179955 -0.34073811 -0.8941051
 -1.02131988 -1.47095915 -1.73785371]
[[14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]
 [ 3  6  2  8 10  5 14  9 13 11  7  1  0 12  4]
 [ 8  9 14  0  5  7  3 12  2  1 10 11  6  4 13]
 [12  9 11  0  8 14  3 10 13  4  5  2  1  6  7]
 [ 6  5 10  0 11  1 12  9 14 13  7  2  4  8  3]]
[[16 15 14 13 12 11 10  9  8  7  6  5  4  3  2]
 [ 5  8  4 10 12  7 16 11 15 13  9  3  2 14  6]
 [10 11 16  2  7  9  5 14  4  3 12 13  8  6 15]
 [14 11 13  2 10 16  5 12 15  6  7  4  3  8  9]
 [ 8  7 12  2 13  3 14 11 16 15  9  4  6 10  5]]
[[ 0.42667739  0.43552045  0.27322318  0.24280688  0.23479485  0.14674026
   0.1414443   0.13735744  0.12593551  0.0148893  -0.1318155  -0.38507011
  -0.51065994 -0.92807189 -1.73785371]
 [ 0.44801457  0.29949741 -0.51065994 -0.5231461   0.4760746   0.0148893
  -0.36773979  0.24331474 -0.22885337  0.11766515 -0.10749091  0.32028385
 

In [ ]:
mf.users.actual_user_scores.value

In [ ]:
print(mf.users.actual_user_scores.shape)
print(mf.predicted_user_item_scores.shape)

actual = np.reshape(mf.users.actual_user_scores.value, (number_of_users*number_of_items, 1))
predicted = np.reshape(mf.predicted_user_item_scores, (number_of_users*number_of_items, 1))
print(test.shape)

plt.hist(actual, alpha=0.7)
plt.hist(predicted, alpha=0.7)#predicted scores are more spread out, which kind of makes sense

In [ ]:
abs_error = abs(actual-predicted)

plt.hist(abs_error)

In [ ]:
# Collect measurements about the simulation
results = mf.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
# Collect measurements about the simulation
results = ideal.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
results = content.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
#generate a distribution that hides a subpopulation
number_of_attributes = 10
number_of_maj_users = 150
number_of_min_users = 50

maj_user_representation = np.random.normal(1, 2, size=(number_of_maj_users, number_of_attributes))
min_user_representation = np.random.normal(0.5, 1.25, size=(number_of_min_users, number_of_attributes))
actual_user_representation = np.vstack((maj_user_representation, min_user_representation))
split_indices=number_of_maj_users

In [ ]:
#If plotted without respect to the subgroups, preference means look more or less normally distributed
plt.hist(actual_user_representation.mean(axis=1))



In [ ]:
#when plotting out mean preferences when accounting for group, we can see a clear distinction in preference
plt.hist(maj_user_representation.mean(axis=1), alpha=.7, color='b')
plt.hist(min_user_representation.mean(axis=1), alpha=0.7, color='r')



In [ ]:
filtering = ContentFiltering(actual_user_representation=actual_user_representation, 
                             num_attributes=number_of_attributes,
                             num_items=500)


mse = MSEMeasurement(diagnostics=True)
recall=RecallMeasurement()

filtering.add_metrics(mse, recall)

filtering.startup_and_train(50)
filtering.run(450)

In [ ]:
mse_diagnostics = filtering.metrics[0].get_diagnostics()
mse_beginning = mse_diagnostics.loc[50:, :]
mse_beginning.head()



In [ ]:
def mse_histogram(model, split_indices=None):
    metric = (
                model.predicted_scores.value.mean(axis=1)- model.users.actual_user_scores.value.mean(axis=1))** 2
    
    colors = ["blue", "orange", "red", "yellow", "green"]
    if split_indices is not None and len(split_indices) > 0:
        splits = [0] + split_indices + [metric.size]
        for i in range(len(splits) - 1):
            values = metric[splits[i] : splits[i + 1]]
            plt.hist(values, alpha=0.7, color=colors[i])
    else:
        plt.hist(metric, bins="auto")
        plt.ylabel("observation count (total n={})".format(metric.size))
        plt.xlabel("mean sqaured error")

In [ ]:
mf.startup_and_train(50)


In [ ]:
mse_histogram(mf)

In [ ]:
mf.run(50)
mf.train()
mse_histogram(mf)

In [ ]:
mf.run(50)
mf.train()
mse_histogram(mf)

In [ ]:
mf.run(50)
mf.train()
mse_histogram(mf)

In [ ]:
content.run(50)
mse_histogram(content)

In [ ]:
content.run(50)
mse_histogram(content)

In [ ]:
print(in_k)
print(not_in_k)
print(len(model.interactions))
k

In [ ]:

np.concatenate((np.ones(len(in_k)), np.zeros(len(not_in_k))), axis=None)

In [ ]:
model=mf
k=5

#split_indices = number_of_maj_users

colors = ["blue", "orange", "red", "yellow", "green"]

shown_item_scores = np.take(model.predicted_scores.value, model.items_shown)
shown_item_ranks = np.argsort(shown_item_scores, axis=1)
top_k_items = np.take(model.items_shown, shown_item_ranks[:, k :])
in_k = (np.where(np.isin(model.interactions, top_k_items))[0])
not_in_k = (np.where(~np.isin(model.interactions, top_k_items))[0])
metric = np.concatenate((np.ones(len(in_k)), np.zeros(len(not_in_k))), axis=None)

plt.hist(metric)

##Amy, implement this pie chart for recall at k
# # Pie chart, where the slices will be ordered and plotted counter-clockwise:
# labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
# sizes = [15, 30, 45, 10]
# explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

# fig1, ax1 = plt.subplots()
# ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
#         shadow=True, startangle=90)
# ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# plt.show()

# maj_population_outcomes = metric[:split_indices]
# min_population_outcomes = metric[split_indices:]

# plt.hist(maj_population_outcomes, color=colors[0])
# plt.hist(min_population_outcomes, color=colors[1])



# if split_indices is not None:
#     splits = [0] + split_indices + [metric.size]
#     for i in range(len(splits) - 1):
#         values = metric[splits[i] : splits[i + 1]]
#         plt.hist(values, alpha=0.7, color=colors[i])

# plt.hist(metric, bins="auto")
# plt.ylabel("observation count (total n={})".format(metric.size))
# plt.xlabel("recall at k")


# 
#     if split_indices is not None and len(split_indices) > 0:
#         splits = [0] + split_indices + [metric.size]
#         for i in range(len(splits) - 1):
#             values = metric[splits[i] : splits[i + 1]]
#             plt.hist(values, alpha=0.7, color=colors[i])
#     else:
#         plt.hist(metric, bins="auto")
#         plt.ylabel("observation count (total n={})".format(metric.size))
#         plt.xlabel("mean sqaured error")

In [ ]:
plt.hist(min_population_outcomes)

In [ ]:
len(min_population_outcomes)

In [ ]:
metric_histogram(filtering)

In [ ]:
metric_histogram(filtering, [number_of_maj_users])

In [ ]:
#generate a bimodal distribution
N=500
mu, sigma = 1.845, 1
mu2, sigma2 = 5.845, 1
X1 = np.random.normal(mu, sigma, N)
X2 = np.random.normal(mu2, sigma2, N)
X_bimodal = np.concatenate([X1, X2])

In [ ]:
# print majority / minority outcome stats
def majority_minority_outcomes(metric, split_index):
    split_indices = [split_index]

        
    maj_mean = metric.last_observation[:split_index].mean()
    maj_std = metric.last_observation[:split_index].std()

    min_mean = metric.last_observation[split_index:].mean()
    min_std = metric.last_observation[split_index:].std()

    print("Majority group statistics: ", maj_mean, "(mean), ", maj_std, "(std)")
    print("Minority group statistics: ", min_mean, "(mean), ", min_std, "(std)")
    print()
    
    metric.hist(split_indices)

In what follows, we expand on this minimal example to gain a deeper understanding of what happens under the hood.

In [ ]:
filtering = ContentFiltering(actual_user_representation=actual_user_representation, 
                             num_attributes=number_of_attributes,
                             num_items=500)


mse = MSEMeasurement(diagnostics=True)
filtering.add_metrics(mse)

filtering.startup_and_train(50)
majority_minority_outcomes(mse, number_of_maj_users)


In [ ]:
filtering.run(450)
majority_minority_outcomes(mse, number_of_maj_users)

In [ ]:
bimodal = plt.hist(X_bimodal, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Bimodal Distribution')
plt.show()

In [ ]:
print(np.mean(X_bimodal))
print(np.std(X_bimodal))

In [ ]:
N=1000
mu, sigma = 14.99, 4
X1 = np.random.normal(mu, sigma, N)
X_skew = np.log2(X1)

skew = plt.hist(X_skew, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Skewed Distribution')
plt.show()

In [ ]:
N=1000
mu, sigma = 3.85, 2.2
normal_dist = np.random.normal(mu, sigma, N)

skew = plt.hist(normal_dist, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Normal Distribution')
plt.show()

In [ ]:
#How to characterize power, type 1 vs type 2 errors 